In [19]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import requests

import hashlib

import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import snap

In [5]:
# Configuración inicial
ruta_lineas = r"C:\Users\AMIM GTAC\Documents\AFC\DSS\Glorieta Minerva\temp\líneas.shp"

output_gpkg_file = r"C:\Users\AMIM GTAC\Documents\AFC\DSS\Glorieta Minerva\temp\líneas_conteo.shp"

In [25]:
# Crear GeoDataFrame de líneas
gdf_lines = gpd.read_file(ruta_lineas)#, crs='EPSG:6368')

gdf_lines = gdf_lines.to_crs(epsg=6368) 

gdf_lines['count'] = 1

# Aplicar una simplificación mínima a las geometrías para reducir diferencias insignificantes
gdf_lines['geometry'] = gdf_lines['geometry'].simplify(tolerance=1)  # Ajusta la tolerancia según necesidad

tolerance = 1 
gdf_lines['geometry'] = gdf_lines['geometry'].apply(lambda geom: snap(geom, geom, tolerance))


# Crear una función para generar un hash basado en la geometría de la línea
def geometry_hash(geometry):
    return hashlib.sha256(geometry.wkb).hexdigest()

# Crear una nueva columna con el hash de la geometría
gdf_lines['geometry_hash'] = gdf_lines['geometry'].apply(geometry_hash)

# Agrupar por el hash de la geometría y contar el número de ocurrencias
gdf_grouped = gdf_lines.groupby('geometry_hash').agg({
    'geometry': 'first',
    'count': 'size'
}).reset_index()

# Renombrar la columna 'count' a 'NUMERO'
gdf_grouped = gdf_grouped.rename(columns={'count': 'NUMERO'})

# Convertir a GeoDataFrame
gdf_grouped = gpd.GeoDataFrame(gdf_grouped, geometry='geometry', crs='EPSG:6368')

# Exportar a geopackage
gdf_grouped.to_file(output_gpkg_file)


C:\Users\AMIM GTAC\AppData\Local\Temp\ipykernel_3308\2876963796.py:35: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_grouped.to_file(output_gpkg_file)
C:\Users\AMIM GTAC\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'geometry_hash' to 'geometry_h'
  ogr_write(


In [23]:
import geopandas as gpd
import hashlib
from shapely.ops import unary_union

# Cargar GeoDataFrame de líneas
gdf_lines = gpd.read_file(ruta_lineas)

# Convertir a un sistema de coordenadas proyectado en metros
gdf_lines = gdf_lines.to_crs(epsg=6368)

# Aplicar un buffer muy pequeño para unir líneas cercanas (ajusta la tolerancia según necesidad)
tolerance = 1  # 1 metro
buffered = gdf_lines.buffer(tolerance)

# Fusionar las líneas superpuestas
merged = unary_union(buffered)

# Convertir el resultado en una nueva capa de líneas (multiparte a líneas individuales)
gdf_grouped = gpd.GeoDataFrame(geometry=gpd.GeoSeries(merged), crs="EPSG:6368")

# Aplicar un buffer negativo para recuperar la geometría original
gdf_grouped["geometry"] = gdf_grouped["geometry"].buffer(-tolerance)

# Crear una función para generar un hash basado en la geometría de la línea
def geometry_hash(geometry):
    return hashlib.sha256(geometry.wkb).hexdigest()

# Crear una nueva columna con el hash de la geometría
gdf_grouped["geometry_hash"] = gdf_grouped["geometry"].apply(geometry_hash)

# Agrupar por geometría única y contar ocurrencias
gdf_grouped = gdf_grouped.groupby("geometry_hash").agg({
    "geometry": "first",
}).reset_index()

# Contar el número de líneas fusionadas
gdf_grouped["NUMERO"] = gdf_grouped.index + 1  # Puedes mejorar esto si tienes datos adicionales

# Convertir a GeoDataFrame
gdf_grouped = gpd.GeoDataFrame(gdf_grouped, geometry="geometry", crs="EPSG:6368")

# Exportar a geopackage
gdf_grouped.to_file(output_gpkg_file)


C:\Users\AMIM GTAC\AppData\Local\Temp\ipykernel_3308\1603043691.py:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_grouped.to_file(output_gpkg_file)
C:\Users\AMIM GTAC\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'geometry_hash' to 'geometry_h'
  ogr_write(
